In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

import models

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new, predict_r0_multipliers, save_r0_mul
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
#from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles, plot_r0_multipliers, plot_daily_trend

import yaml

In [ ]:
predictions_dict = {}

In [ ]:
# config_filename = 'jharkhand.yaml'
config_filename = 'ranchi.yaml'
# config_filename = 'east singhbhum.yaml'
# config_filename = 'dhanbad.yaml'
# config_filename = 'bokaro.yaml'
config = read_config(config_filename)

## Perform M0 fit

In [ ]:
m0_params = copy.deepcopy(config['fitting'])
if m0_params['data']['data_source'] == 'athena':
    config['fitting']['data']['data_source'] = 'covid19india'
    m0_tag = 'athena'
    m1_tag = 'covid19india'
else:
    config['fitting']['data']['data_source'] = 'athena'
    m0_tag = 'covid19india'
    m1_tag = 'athena'
m1_params = copy.deepcopy(config['fitting'])

In [ ]:
predictions_dict[m0_tag] = single_fitting_cycle(**m0_params)

In [ ]:
predictions_dict[m1_tag] = single_fitting_cycle(**m1_params)

In [ ]:
predictions_dict[m0_tag]['df_district']

In [ ]:
predictions_dict[m1_tag]['df_district']

In [ ]:
compartments_to_plot = ['total', 'active', 'recovered', 'deceased']
colors = "bygr"
markers = 'ox'
fig = plt.figure(figsize=(20,20))
for i,compartment in enumerate(compartments_to_plot):
    for j, tag in enumerate(predictions_dict):
        plt.plot(predictions_dict[tag]['df_district']['date'], predictions_dict[tag]['df_district'][compartment], marker=markers[j], color=colors[i], label=compartment+ ' ' + tag)
plt.legend(loc='best', fontsize=20)
plt.xlabel('date', fontsize=20)
plt.ylabel('count', fontsize=20)
plt.savefig("../../misc/reports/comparison_" + config_filename.split('.')[0] + '.png')